<a href="https://colab.research.google.com/github/rishi15-t/PP-MM/blob/master/PP_MM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
!pip install git+https://github.com/uber/pyro.git
#dataset = pd.read_pickle('/content/drive/My Drive/dataset/dataset_gmu_paper.pkl')

  Cloning https://github.com/uber/pyro.git to /tmp/pip-req-build-6uuvtpsp
  Running command git clone -q https://github.com/uber/pyro.git /tmp/pip-req-build-6uuvtpsp
     |████████████████████████████████| 61kB 2.8MB/s 
  Created wheel for pyro-ppl: filename=pyro_ppl-1.2.1+a11f1110-cp36-none-any.whl size=494336 sha256=39dcc90eb40005aaacce9d74e8171eaddf2383f4ded72f2cc733bf8d959f2a72
  Stored in directory: /tmp/pip-ephem-wheel-cache-dlzjito2/wheels/3c/10/ed/73d2332a097e2e9dc7d601ca2a99b0b4f9634e393474b78137
Successfully built pyro-ppl
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [3]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss
from torch.nn.functional import softplus
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils import clip_grad_norm_
from torch.optim import AdamW
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm, trange
from sklearn import metrics
#!pip install transformers
#from transformers import get_linear_schedule_with_warmup
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
import time
import datetime
#!pip install git+https://github.com/uber/pyro.git
import pyro
from pyro import poutine
from pyro.distributions import Normal, Categorical, Laplace
from pyro.infer import SVI, Trace_ELBO, TraceEnum_ELBO
from pyro.optim import ClippedAdam


'''def Train_Test_Val_Split(data , test_data_fraction = 0.3, val_data_fraction = 0.1) :
    
    mlb = MultiLabelBinarizer()
    data_genres_one_hot_encoding = mlb.fit_transform(data['genres'])
    Label_names = mlb.classes_
    data_genres_one_hot_encoding = pd.DataFrame(data_genres_one_hot_encoding, columns = mlb.classes_)
    Data_train, Data_test, Labels_train, Labels_test = train_test_split(data, data_genres_one_hot_encoding, test_size = test_data_fraction)
    Labels_train = torch.tensor(Labels_train.values)
    Labels_test = torch.tensor(Labels_test.values)
    
    Data_train, Data_val, Labels_train, Labels_val = train_test_split(Data_train, Labels_train, test_size = val_data_fraction)

    Data_train = Data_train.reset_index(drop=True)
    Data_test = Data_test.reset_index(drop=True)
    Data_val = Data_val.reset_index(drop=True)
    

    return (Data_train, Data_test, Data_val, Labels_train, Labels_test, Labels_val, Label_names)
    
Data_train, Data_test, Data_val, Labels_train_tensor, Labels_test_tensor, Labels_val_tensor, Label_names = Train_Test_Val_Split(dataset)


Data_train_tensor_text = torch.tensor(Data_train['w2v_embeddings'])
Data_test_tensor_text = torch.tensor(Data_test['w2v_embeddings'])
Data_val_tensor_text = torch.tensor(Data_val['w2v_embeddings'])

Data_train_tensor_image = torch.tensor(Data_train['vgg16_embeddings'])
Data_test_tensor_image = torch.tensor(Data_test['vgg16_embeddings'])
Data_val_tensor_image = torch.tensor(Data_val['vgg16_embeddings'])'''

"def Train_Test_Val_Split(data , test_data_fraction = 0.3, val_data_fraction = 0.1) :\n    \n    mlb = MultiLabelBinarizer()\n    data_genres_one_hot_encoding = mlb.fit_transform(data['genres'])\n    Label_names = mlb.classes_\n    data_genres_one_hot_encoding = pd.DataFrame(data_genres_one_hot_encoding, columns = mlb.classes_)\n    Data_train, Data_test, Labels_train, Labels_test = train_test_split(data, data_genres_one_hot_encoding, test_size = test_data_fraction)\n    Labels_train = torch.tensor(Labels_train.values)\n    Labels_test = torch.tensor(Labels_test.values)\n    \n    Data_train, Data_val, Labels_train, Labels_val = train_test_split(Data_train, Labels_train, test_size = val_data_fraction)\n\n    Data_train = Data_train.reset_index(drop=True)\n    Data_test = Data_test.reset_index(drop=True)\n    Data_val = Data_val.reset_index(drop=True)\n    \n\n    return (Data_train, Data_test, Data_val, Labels_train, Labels_test, Labels_val, Label_names)\n    \nData_train, Data_test, D

In [0]:
Data_train_tensor_text = torch.load("/content/drive/My Drive/dataset/Data_train_tensor_text.pt")
Data_test_tensor_text = torch.load( "/content/drive/My Drive/dataset/Data_test_tensor_text.pt")
Data_val_tensor_text = torch.load( "/content/drive/My Drive/dataset/Data_val_tensor_text.pt")
Data_train_tensor_image = torch.load( "/content/drive/My Drive/dataset/Data_train_tensor_image.pt")
Data_test_tensor_image = torch.load( "/content/drive/My Drive/dataset/Data_test_tensor_image.pt")
Data_val_tensor_image = torch.load( "/content/drive/My Drive/dataset/Data_val_tensor_image.pt")
Labels_train_tensor = torch.load("/content/drive/My Drive/dataset/Labels_train_tensor.pt")
Labels_test_tensor = torch.load("/content/drive/My Drive/dataset/Labels_test_tensor.pt")
Labels_val_tensor = torch.load("/content/drive/My Drive/dataset/Labels_val_tensor.pt")

Label_names = np.array(['Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
       'History', 'Horror', 'Music', 'Musical', 'Mystery', 'Romance',
       'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War', 'Western'],
      dtype=object)

In [0]:
#Neural networks


#source: https://github.com/Duncanswilson/maxout-pytorch/blob/master/maxout_pytorch.ipynb
class ListModule(object):
    def __init__(self, module, prefix, *args):
        self.module = module
        self.prefix = prefix
        self.num_module = 0
        for new_module in args:
            self.append(new_module)

    def append(self, new_module):
        if not isinstance(new_module, nn.Module):
            raise ValueError('Not a Module')
        else:
            self.module.add_module(self.prefix + str(self.num_module), new_module)
            self.num_module += 1

    def __len__(self):
        return self.num_module

    def __getitem__(self, i):
        if i < 0 or i >= self.num_module:
            raise IndexError('Out of bound')
        return getattr(self.module, self.prefix + str(i))


class Maxout_MLP(nn.Module):
    
    def __init__(self, hidden_layer_size1, hidden_layer_size2, dropout, num_labels = 23, num_maxout_units=2):
        
        super(Maxout_MLP, self).__init__()
        self.fc1_list = ListModule(self, "fc1_")
        self.fc2_list = ListModule(self, "fc2_")
        self.hidden_layer_size1 = hidden_layer_size1
        self.hidden_layer_size2 = hidden_layer_size2
        for _ in range(num_maxout_units):
            self.fc1_list.append(nn.Linear(self.hidden_layer_size1, self.hidden_layer_size2))
            self.fc2_list.append(nn.Linear(self.hidden_layer_size2, self.hidden_layer_size2))
        self.dropout = nn.Dropout(p=dropout)
        self.sigmoid = torch.nn.Sigmoid()
        self.linear = torch.nn.Linear(self.hidden_layer_size2, num_labels)  #PP_MM_pattern 4

        self.bn0 = nn.BatchNorm1d(self.hidden_layer_size1)
        self.bn1 = nn.BatchNorm1d(self.hidden_layer_size2)
        self.bn2 = nn.BatchNorm1d(self.hidden_layer_size2)

    def forward(self, x): 
        
        x = x.view(-1, self.hidden_layer_size1)
        x = self.bn0(x)
        x = self.maxout(x, self.fc1_list)
        x = self.bn1(x)
        x = self.dropout(x)
        x = self.maxout(x, self.fc2_list)
        x = self.bn2(x)
        '''
        PP_MM_pattern 4
        '''
        logits = self.linear(x)  
        if(self.training) :     
            return logits
        else :
            output = self.sigmoid(logits)
            return output

    def maxout(self, x, layer_list):
        
        max_output = layer_list[0](x)
        for _, layer in enumerate(layer_list, start=1):
            max_output = torch.max(max_output, layer(x))
        return max_output


class GMU(nn.Module):

    def __init__(self, num_maxout_units = 2, hidden_layer_size = 512, text_embeddings_size = 300, img_embeddings_size = 4096, num_labels = 23, hidden_activation = None, dropout = 0.1):

        super(GMU, self).__init__()
        self.num_labels = num_labels
        self.hidden_layer_size = hidden_layer_size

        self.linear_h_text = torch.nn.Linear(text_embeddings_size, self.hidden_layer_size)
        self.linear_h_image = torch.nn.Linear(img_embeddings_size, self.hidden_layer_size)
        self.linear_z = torch.nn.Linear(text_embeddings_size + img_embeddings_size, self.hidden_layer_size)
       
        self.tanh = torch.nn.Tanh()
        self.sigmoid = torch.nn.Sigmoid()
        self.dropout = nn.Dropout(p=dropout)

        self.bn0 = nn.BatchNorm1d(img_embeddings_size)
        self.bn1 = nn.BatchNorm1d(text_embeddings_size)
        self.bn2 = nn.BatchNorm1d(text_embeddings_size + img_embeddings_size)

    def forward(self, text_embeddings, image_embeddings):
        
        image_embeddings = self.bn0(image_embeddings)
        image_h = self.linear_h_image(image_embeddings)
        image_h = self.tanh(image_h)

        text_embeddings = self.bn1(text_embeddings)
        text_h = self.linear_h_text(text_embeddings)
        text_h = self.tanh(text_h)

        concat = torch.cat((image_embeddings, text_embeddings), 1)
        concat = self.bn2(concat)
        z = self.linear_z(concat)
        z = self.sigmoid(z)
        gmu_output = z*image_h + (1-z)*text_h
        
        attendout = self.tanh(gmu_output)  

        return attendout   


 

In [0]:
# Pyro module defined only over the GMU NN

class GMU_Pyro():

    def __init__(self, model):

        self.model = model 
    
    def gmu_model(self, text_embeddings, image_embeddings, labels):

        pyro.module("GMU", self.model)
        #global x_label

        linear_h_text_prior_w = Laplace(loc=torch.ones_like(self.model.linear_h_text.weight), scale=torch.ones_like(self.model.linear_h_text.weight))
        linear_h_text_prior_b = Laplace(loc=torch.ones_like(self.model.linear_h_text.bias), scale=torch.ones_like(self.model.linear_h_text.bias))
        linear_h_image_prior_w = Laplace(loc=torch.ones_like(self.model.linear_h_image.weight), scale=torch.ones_like(self.model.linear_h_image.weight))
        linear_h_image_prior_b = Laplace(loc=torch.ones_like(self.model.linear_h_image.bias), scale=torch.ones_like(self.model.linear_h_image.bias))
        linear_z_prior_w = Laplace(loc=torch.ones_like(self.model.linear_z.weight), scale=torch.ones_like(self.model.linear_z.weight))
        linear_z_prior_b = Laplace(loc=torch.ones_like(self.model.linear_z.bias), scale=torch.ones_like(self.model.linear_z.bias))

        priors = {
            'linear_h_text.weight': linear_h_text_prior_w, 'linear_h_text.bias': linear_h_text_prior_b,
            'linear_h_image.weight': linear_h_image_prior_w, 'linear_h_image.bias': linear_h_image_prior_b,
            'linear_z.weight': linear_z_prior_w, 'linear_z.bias': linear_z_prior_b
            }

        lifted_module = pyro.random_module("module", self.model, priors)
        lifted_reg_model = lifted_module()

        lhat = torch.sigmoid(lifted_reg_model(text_embeddings, image_embeddings))
        pyro.sample("obs", Categorical(logits=lhat), obs=labels)

    
    def gmu_guide(self, text_embeddings, image_embeddings, labels):

        pyro.module("GMU", self.model)

        # linear_h_text
        # weight
        lwlinear_h_text = torch.empty_like(self.model.linear_h_text.weight)
        swlinear_h_text = torch.empty_like(self.model.linear_h_text.weight)
        torch.nn.init.normal_(lwlinear_h_text, std=0.001)
        torch.nn.init.normal_(swlinear_h_text, std=0.01)
        linear_h_text_loc_param_w_l = pyro.param("linear_h_text_loc_w_l", lwlinear_h_text)
        linear_h_text_loc_param_w_s = softplus(pyro.param("linear_h_text_loc_w_s", swlinear_h_text))
        linear_h_text_prior_w = Laplace(loc=linear_h_text_loc_param_w_l, scale=linear_h_text_loc_param_w_s)
        # bias
        lblinear_h_text = torch.empty_like(self.model.linear_h_text.bias)
        sblinear_h_text = torch.empty_like(self.model.linear_h_text.bias)
        torch.nn.init.normal_(lblinear_h_text, std=0.001)
        torch.nn.init.normal_(sblinear_h_text, std=0.01)
        linear_h_text_loc_param_b_l = pyro.param("linear_h_text_loc_b_l", lblinear_h_text)
        linear_h_text_loc_param_b_s = softplus(pyro.param("linear_h_text_loc_b_s", sblinear_h_text))
        linear_h_text_prior_b = Laplace(loc=linear_h_text_loc_param_b_l, scale=linear_h_text_loc_param_b_s)

        # linear_h_image
        # weight
        lwlinear_h_image = torch.empty_like(self.model.linear_h_image.weight)
        swlinear_h_image = torch.empty_like(self.model.linear_h_image.weight)
        torch.nn.init.normal_(lwlinear_h_image, std=0.001)
        torch.nn.init.normal_(swlinear_h_image, std=0.01)
        linear_h_image_loc_param_w_l = pyro.param("linear_h_image_loc_w_l", lwlinear_h_image)
        linear_h_image_loc_param_w_s = softplus(pyro.param("linear_h_image_loc_w_s", swlinear_h_image))
        linear_h_image_prior_w = Laplace(loc=linear_h_image_loc_param_w_l, scale=linear_h_image_loc_param_w_s)
        # bias
        lblinear_h_image = torch.empty_like(self.model.linear_h_image.bias)
        sblinear_h_image = torch.empty_like(self.model.linear_h_image.bias)
        torch.nn.init.normal_(lblinear_h_image, std=0.001)
        torch.nn.init.normal_(sblinear_h_image, std=0.01)
        linear_h_image_loc_param_b_l = pyro.param("linear_h_image_loc_b_l", lblinear_h_image)
        linear_h_image_loc_param_b_s = softplus(pyro.param("linear_h_image_loc_b_s", sblinear_h_image))
        linear_h_image_prior_b = Laplace(loc=linear_h_image_loc_param_b_l, scale=linear_h_image_loc_param_b_s)

        # linear_z
        # weight
        lwlinear_z = torch.empty_like(self.model.linear_z.weight)
        swlinear_z = torch.empty_like(self.model.linear_z.weight)
        torch.nn.init.normal_(lwlinear_z, std=0.001)
        torch.nn.init.normal_(swlinear_z, std=0.01)
        linear_z_loc_param_w_l = pyro.param("linear_z_loc_w_l", lwlinear_z)
        linear_z_loc_param_w_s = softplus(pyro.param("linear_z_loc_w_s", swlinear_z))
        linear_z_prior_w = Laplace(loc=linear_z_loc_param_w_l, scale=linear_z_loc_param_w_s)
        # bias
        lblinear_z = torch.empty_like(self.model.linear_z.bias)
        sblinear_z = torch.empty_like(self.model.linear_z.bias)
        torch.nn.init.normal_(lblinear_z, std=0.001)
        torch.nn.init.normal_(sblinear_z, std=0.01)
        linear_z_loc_param_b_l = pyro.param("linear_z_loc_b_l", lblinear_z)
        linear_z_loc_param_b_s = softplus(pyro.param("linear_z_loc_b_s", sblinear_z))
        linear_z_prior_b = Laplace(loc=linear_z_loc_param_b_l, scale=linear_z_loc_param_b_s)

        priors = {
        'linear_h_text.weight': linear_h_text_prior_w, 'linear_h_text.bias': linear_h_text_prior_b,
        'linear_h_image.weight': linear_h_image_prior_w, 'linear_h_image.bias': linear_h_image_prior_b,
        'linear_z.weight': linear_z_prior_w, 'linear_z.bias': linear_z_prior_b
        }

        lifted_module = pyro.random_module("module", self.model, priors)

        return lifted_module() 

In [0]:
#Train-Validation-Test

class Training_Testing_MM():

    def __init__(self, Data_train_tensor_text, Data_train_tensor_image, Labels_train_tensor, 
                 Data_test_tensor_text, Data_test_tensor_image, Labels_test_tensor, 
                 Data_val_tensor_text, Data_val_tensor_image, Labels_val_tensor,
                 Label_names = None, hidden_layer_size = 512, num_maxout_units = 2, weight_decay= 0.1, reg_param = 0.1, scheduler_step_size = 30, scheduler_lr_fraction = 0.8,
                 hidden_activation = "tanh", batch_size = 32, epochs = 10, sigmoid_thresh = 0.2, learning_rate = 2e-5, num_labels = 23, dropout = 0.1, max_norm = 5):


      self.gmu = GMU(num_maxout_units = num_maxout_units, hidden_layer_size = hidden_layer_size, hidden_activation = hidden_activation, dropout = dropout).cuda()
      self.mlp = Maxout_MLP(hidden_layer_size, hidden_layer_size, dropout = dropout, num_labels = num_labels, num_maxout_units = num_maxout_units).cuda()
      self.pyro = GMU_Pyro(model = self.gmu)
      #self.inference = SVI(self.pyro.gmu_model, self.pyro.gmu_guide, ClippedAdam({"lr": learning_rate}), loss = self.custom_mc_elbo)
      #self.inference = SVI(self.pyro.gmu_model, self.pyro.gmu_guide, ClippedAdam({"lr": learning_rate}), loss = Trace_ELBO())
      #self.inference = SVI(self.pyro.gmu_model, self.pyro.gmu_guide, ClippedAdam({"lr": learning_rate}), loss = self.simple_elbo_kl_annealing)
      self.inference = SVI(self.pyro.gmu_model, self.pyro.gmu_guide, ClippedAdam({"lr": learning_rate}), loss = self.custom_elbo_with_wd)
      self.label_names = Label_names
      self.num_labels = num_labels
      self.batch_size = batch_size
      self.learning_rate = learning_rate
      self.max_norm = max_norm
      self.epochs = epochs
      self.sigmoid_thresh = sigmoid_thresh
      self.scheduler_step_size = scheduler_step_size
      self.scheduler_lr_fraction = scheduler_lr_fraction
      self.weight_decay = weight_decay
      self.reg_param = reg_param
      self.optimizer = self.SetOptimizer()
      self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
      self.results = pd.DataFrame(0, index=['Recall','Precision','F_Score'], columns=['micro', 'macro', 'weighted', 'samples']).astype(float)
      self.epoch_loss_set = []
      self.epoch_gmu_loss_set = []
      self.train_dataloader = self.SetTrainDataloader_MM(Data_train_tensor_text, Data_train_tensor_image, Labels_train_tensor)
      self.test_dataloader = self.SetTestDataloader_MM(Data_test_tensor_text, Data_test_tensor_image, Labels_test_tensor) 
      self.scheduler = self.SetScheduler()

      self.val_accuracy_set = [] 
      self.val_dataloader = self.SetValDataloader_MM(Data_val_tensor_text, Data_val_tensor_image, Labels_val_tensor)
      self.class_wise_metrics = None
      self.predictions = None


    # custom Elbo
    def custom_mc_elbo(self, model, guide, *args):
      guide_trace = poutine.trace(guide).get_trace(*args)
      model_trace = poutine.trace(poutine.replay(model, trace=guide_trace)).get_trace(*args)
      elbo = model_trace.log_prob_sum() - guide_trace.log_prob_sum()
      return -elbo


    # custom Elbo with WD
    def custom_elbo_with_wd(self, model, guide, *args):
        guide_trace = poutine.trace(guide).get_trace(*args)
        model_trace = poutine.trace(poutine.replay(model, trace=guide_trace)).get_trace(*args)
        logm = model_trace.log_prob_sum()
        logg = guide_trace.log_prob_sum()
        wds = 0.01
        wd = 0.
        for node in model_trace.nodes.values():
            if node["type"] == "param":
                wd = wd + wds * torch.sum(node["value"])
        for node in guide_trace.nodes.values():
            if node["type"] == "param":
                wd = wd + wds * torch.sum(node["value"])
        #mixmod_trace = mmd(model_trace, guide_trace)
        elbo = logm - logg + wd
        return -elbo


    def simple_elbo_kl_annealing(self, model, guide, *args):
        # get the annealing factor and latents to anneal from the keyword
        # arguments passed to the model and guide
        annealing_factor = 2.4
        latents_to_anneal = ["my_latent"]
        # run the guide and replay the model against the guide
        guide_trace = poutine.trace(guide).get_trace(*args)
        model_trace = poutine.trace(poutine.replay(model, trace=guide_trace)).get_trace(*args)

        elbo = 0.0
        # loop through all the sample sites in the model and guide trace and
        # construct the loss; note that we scale all the log probabilities of
        # samples sites in `latents_to_anneal` by the factor `annealing_factor`
        for site in model_trace.nodes.values():
            if site["type"] == "sample":
                factor = annealing_factor if site["name"] in latents_to_anneal else 1.0
                elbo = elbo + factor * site["fn"].log_prob(site["value"]).sum()
        for site in guide_trace.nodes.values():
            if site["type"] == "sample":
                factor = annealing_factor if site["name"] in latents_to_anneal else 1.0
                elbo = elbo - factor * site["fn"].log_prob(site["value"]).sum()
        return -elbo


    def L2_Regularizer(self):
      reg_loss = 0.0
      for param in self.gmu.parameters():
          reg_loss = reg_loss + param.pow(2.0).sum()
      return self.reg_param*reg_loss


    def SetOptimizer(self) :

      optimizer = AdamW(self.mlp.parameters(), lr=self.learning_rate,  eps = 1e-6, weight_decay=self.weight_decay)
      #optimizer = Adam(self.mlp.parameters(), lr=self.learning_rate,  eps = 1e-6, weight_decay=self.weight_decay)
      return(optimizer)

    

    def SetScheduler(self) :

      '''
      scheduler = get_linear_schedule_with_warmup(self.optimizer, num_warmup_steps = 10, 
                                                 num_training_steps = self.epochs*len(self.train_dataloader))
      '''
      scheduler = StepLR(self.optimizer, step_size = self.scheduler_step_size, gamma = self.scheduler_lr_fraction)
      return(scheduler) 



    def Get_Metrics(self, actual, predicted) :

      #acc = metrics.accuracy_score(actual, predicted)
      #hamming = metrics.hamming_loss(actual, predicted)
      #(metrics.roc_auc_score(actual, predicted, average=average)
      averages = ('micro', 'macro', 'weighted', 'samples')
      for average in averages:
          precision, recall, fscore, _ = metrics.precision_recall_fscore_support(actual, predicted, average=average)
          self.results[average]['Recall'] += recall
          self.results[average]['Precision'] += precision
          self.results[average]['F_Score'] += fscore



    #source: https://mccormickml.com/2019/07/22/BERT-fine-tuning/
    def Plot_Training_Epoch_Loss(self) :

      sns.set(style='darkgrid')
      sns.set(font_scale=1.5)
      plt.rcParams["figure.figsize"] = (12,6)
      plt.plot(self.epoch_loss_set, 'b-o')
      plt.title("Training loss")
      plt.xlabel("Epoch")
      plt.ylabel("Loss")
      plt.savefig('Training_Epoch_Loss.png',bbox_inches='tight')
      plt.show()


    def Plot_Training_Epoch_SVI_Loss(self) :

      sns.set(style='darkgrid')
      sns.set(font_scale=1.5)
      plt.rcParams["figure.figsize"] = (12,6)
      plt.plot(self.epoch_gmu_loss_set, 'b-o')
      plt.title("Training loss (SVI)")
      plt.xlabel("Epoch")
      plt.ylabel("Loss")
      plt.savefig('Training_Epoch_SVI_Loss.png',bbox_inches='tight')
      plt.show()

    
    def Plot_Training_Epoch_Accuracy(self) :

      sns.set(style='darkgrid')
      sns.set(font_scale=1.5)
      plt.rcParams["figure.figsize"] = (12,6)
      plt.plot(self.val_accuracy_set, 'b-o')
      plt.title("Weighted F1 Score")
      plt.xlabel("Epoch")
      plt.ylabel("Validation Accuracy")
      plt.savefig('Training_Validation_Accuracy.png',bbox_inches='tight')
      plt.show()


    #source: https://mccormickml.com/2019/07/22/BERT-fine-tuning/
    def format_time(self, elapsed):
      '''
      Takes a time in seconds and returns a string hh:mm:ss
      '''
      # Round to the nearest second.
      elapsed_rounded = int(round((elapsed)))
      return str(datetime.timedelta(seconds=elapsed_rounded))


    def SetTrainDataloader_MM(self, Data_train_tensor_text, Data_train_tensor_image, Labels_train_tensor) :

      train_dataset = TensorDataset(Data_train_tensor_text, Data_train_tensor_image, Labels_train_tensor)
      train_sampler = RandomSampler(train_dataset)
      train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size = self.batch_size)
      return(train_dataloader)


    def SetTestDataloader_MM(self, Data_test_tensor_text, Data_test_tensor_image, Labels_test_tensor) :
      
      test_dataset = TensorDataset(Data_test_tensor_text, Data_test_tensor_image, Labels_test_tensor)
      test_sampler = SequentialSampler(test_dataset)
      #test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size = self.batch_size)
      test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size = Data_test_tensor_text.shape[0])
      return(test_dataloader)

    
    def SetValDataloader_MM(self, Data_val_tensor_text, Data_val_tensor_image, Labels_val_tensor) :
      
      val_dataset = TensorDataset(Data_val_tensor_text, Data_val_tensor_image, Labels_val_tensor)
      val_sampler = SequentialSampler(val_dataset)
      #test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size = self.batch_size)
      val_dataloader = DataLoader(val_dataset, sampler=val_sampler, batch_size = Data_val_tensor_text.shape[0])
      return(val_dataloader)

   
    def Train(self) :
      
      # clear param store
      pyro.clear_param_store()
      
      for _ in trange(self.epochs, desc="Epoch"):
        
        self.gmu.train()
        self.mlp.train()
        epoch_loss = 0
        epoch_gmu_loss = 0

        # Measure how long the training epoch takes.
        t0 = time.time()
    
        for step_num, batch_data in enumerate(self.train_dataloader):

          # Progress update every 30 batches.
          if step_num % 30 == 0 and not step_num == 0:
            elapsed = self.format_time(time.time() - t0)
            print('  Batch : ',step_num, ' , Time elapsed : ',elapsed)

          samples_text, samples_image, labels = tuple(t.to(self.device) for t in batch_data)
          self.optimizer.zero_grad()

          ##### Pyro - GMU #####
          gmu_loss = self.inference.step(samples_text.float(), samples_image.float(), labels.t()) #+ self.L2_Regularizer()
          attendout = self.gmu(samples_text, samples_image)
          #epoch_gmu_loss += gmu_loss.detach().cpu().numpy()
          epoch_gmu_loss += gmu_loss

          ##### MLP ####
          logits = self.mlp(attendout)
          loss_fct = BCEWithLogitsLoss()
          batch_loss = loss_fct(logits.view(-1, self.num_labels).float(), labels.view(-1, self.num_labels).float())
          batch_loss.backward()
          clip_grad_norm_(self.mlp.parameters(), norm_type = 2, max_norm = self.max_norm)
          self.optimizer.step()
          self.scheduler.step()
          epoch_loss += batch_loss.item()

        avg_epoch_loss = epoch_loss/len(self.train_dataloader)
        avg_epoch_gmu_loss = epoch_gmu_loss/len(self.train_dataloader)
        print("\nTrain loss for epoch: ",avg_epoch_loss)
        print("\nTrain loss for epoch (gmu): ",avg_epoch_gmu_loss)
        print("\nTraining epoch took: {:}".format(self.format_time(time.time() - t0)))
        self.epoch_loss_set.append(avg_epoch_loss)
        self.epoch_gmu_loss_set.append(avg_epoch_gmu_loss)

        
        #Validation on the epoch
        self.mlp.eval()
        self.gmu.eval()
        epoch_f1_score = 0

        for batch_data in self.val_dataloader:
          samples_text, samples_image, labels = tuple(t.to(self.device) for t in batch_data)
          with torch.no_grad():
            attendout = self.gmu(samples_text.float(), samples_image.float())
            output = self.mlp(attendout)

          threshold = torch.Tensor([self.sigmoid_thresh]).to(self.device)
          predictions = (output > threshold).int()

          predictions = predictions.detach().cpu().numpy()
          labels = labels.to('cpu').numpy()
      
          weighted_f_score = metrics.f1_score(labels,predictions,average="weighted")
          epoch_f1_score += weighted_f_score

        avg_val_f1_score = epoch_f1_score/len(self.val_dataloader)
        print("\nWeighted F1 score for epoch: ",avg_val_f1_score,"\n")
        self.val_accuracy_set.append(avg_val_f1_score)
        

      #torch.save(self.mlp.state_dict(), "/content/drive/My Drive/dataset/model.pt")
      self.Plot_Training_Epoch_Loss()
      self.Plot_Training_Epoch_SVI_Loss()
      self.Plot_Training_Epoch_Accuracy()
   

    def Test(self) :

      # Put model in evaluation mode to evaluate loss on the test set
      self.mlp.eval()
      self.gmu.eval()

      for batch_data in self.test_dataloader:
  
        samples_text, samples_image, labels = tuple(t.to(self.device) for t in batch_data)
      
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        # Forward pass, calculate logit predictions
        with torch.no_grad():
            attendout = self.gmu(samples_text.float(), samples_image.float())
            output = self.mlp(attendout)

        threshold = torch.Tensor([self.sigmoid_thresh]).to(self.device)
        predictions = (output > threshold).int()

        # Move preds and labels to CPU
        predictions = predictions.detach().cpu().numpy()
        labels = labels.to('cpu').numpy()

        self.predictions = predictions
        self.Get_Metrics(labels, predictions)
        self.class_wise_metrics = metrics.classification_report(labels, predictions, target_names= list(self.label_names))
        
    
      self.results = self.results/len(self.test_dataloader)
      #print("Test data metrics : \n")

      #print("\nGenres with no predicted samples : ", self.label_names[np.where(np.sum(predictions, axis=0) == 0)[0]])
      
      return(self.results)

In [8]:
import random
import numpy as np


epochs = 25
hidden_layer_size = 3000
batch_size = 512
learning_rate = 0.005
dropout = 0.9
sigmoid_thresh = 0.3
weight_decay = 0.01
num_maxout_units = 5
max_norm = 20
reg_param = 0.1

hyperparameters = {'hidden_layer_size' : hidden_layer_size, 'epochs' : epochs, 'batch_size' : batch_size, 'learning_rate' : learning_rate, 'dropout' : dropout, 'scheduler_step_size' : 99999, 
                    'scheduler_lr_fraction' : 0.85, 'sigmoid_thresh' : sigmoid_thresh, 'num_maxout_units' : num_maxout_units, 'weight_decay' : weight_decay, 'max_norm' : max_norm, 'reg_param': reg_param}
for key, value in hyperparameters.items():
  print(key," : ",value)


train_test = Training_Testing_MM( Data_train_tensor_text, Data_train_tensor_image, Labels_train_tensor, 
                                  Data_test_tensor_text, Data_test_tensor_image, Labels_test_tensor, 
                                  Data_val_tensor_text, Data_val_tensor_image, Labels_val_tensor, Label_names=Label_names, 
                                  hidden_layer_size = hidden_layer_size, epochs = epochs, batch_size= batch_size, learning_rate = learning_rate, dropout = dropout, scheduler_step_size = 99999, 
                                  scheduler_lr_fraction = 0.85, sigmoid_thresh = sigmoid_thresh, num_maxout_units = num_maxout_units, weight_decay = weight_decay, max_norm = max_norm, reg_param=reg_param)
train_test.Train()
train_test.Test()


hidden_layer_size  :  3000
epochs  :  25
batch_size  :  512
learning_rate  :  0.005
dropout  :  0.9
scheduler_step_size  :  99999
scheduler_lr_fraction  :  0.85
sigmoid_thresh  :  0.3
num_maxout_units  :  5
weight_decay  :  0.01
max_norm  :  20
reg_param  :  0.1


Epoch:   0%|          | 0/25 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/pyro/primitives.py:406: FutureWarning: The `random_module` primitive is deprecated, and will be removed in a future release. Use `pyro.nn.Module` to create Bayesian modules from `torch.nn.Module` instances.
  "modules from `torch.nn.Module` instances.", FutureWarning)


  Batch :  30  , Time elapsed :  0:00:15

Train loss for epoch:  0.6276457961648703

Train loss for epoch (gmu):  12409496.34375

Training epoch took: 0:00:16


Epoch:   4%|▍         | 1/25 [00:16<06:34, 16.45s/it]


Weighted F1 score for epoch:  0.22138217853168515 

  Batch :  30  , Time elapsed :  0:00:15

Train loss for epoch:  0.41445751208812

Train loss for epoch (gmu):  9369641.46875

Training epoch took: 0:00:16


...

Epoch:  64%|██████▍   | 16/25 [04:17<02:24, 16.10s/it]


Weighted F1 score for epoch:  0.24753462568483914 

  Batch :  30  , Time elapsed :  0:00:15

Train loss for epoch:  0.29760390520095825

Train loss for epoch (gmu):  -1549968.58203125

Training epoch took: 0:00:16


Epoch:  68%|██████▊   | 17/25 [04:33<02:08, 16.10s/it]


Weighted F1 score for epoch:  0.07064356016660218 



KeyboardInterrupt: ignored